In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

2023-05-12 22:28:33.997942: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 22:28:34.035904: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 22:28:34.036729: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 22:28:34.683390: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("Y", Y.shape)
print("R", R.shape)
print("num_features", num_features)
print("num_movies",   num_movies)
print("num_users",    num_users)

X (4778, 10)
W (443, 10)
b (1, 443)
Y (4778, 443)
R (4778, 443)
num_features 10
num_movies 4778
num_users 443


Cost Function
$$J= \left[ \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+ \underbrace{\left[
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
\right]}_{regularization}$$

In [3]:
def cofi_cost_func(X, W, b, Y, R, lambda_):

    nm, nu = Y.shape
    J = 0

    for j in range(nu):
        w_j = W[j]
        b_j = b[0, j]
        for i in range(nm):
            x = X[i]
            r = R[i, j]
            J += r*((np.dot(w_j, x)+b_j-Y[i, j])**2)
    J = J/2
    J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))
    return J

J = cofi_cost_func(X, W, b, Y, R, 1.5)
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 306504.87


In [4]:
def cofi_cost_func_vectorized(X, W, b, Y, R, lambda_):
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * \
        (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

J = cofi_cost_func_vectorized(X, W, b, Y, R, 1.5)
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 306504.87


Normalization of ratings

In [5]:
def normalizeRatings(Y, R):
    Ymean = (np.sum(Y*R, axis=1)/(np.sum(R, axis=1)+1e-12)).reshape(-1, 1)
    Ynorm = Y - np.multiply(Ymean, R)
    return (Ynorm, Ymean)

Adding My ratings to the movies

In [6]:
my_ratings = np.zeros(num_movies)
my_ratings[2700] = 5
my_ratings[2609] = 2
my_ratings[929] = 5
my_ratings[246] = 5
my_ratings[2716] = 3
my_ratings[1150] = 5
my_ratings[382] = 2
my_ratings[366] = 5
my_ratings[622] = 5
my_ratings[988] = 3
my_ratings[2925] = 1
my_ratings[2937] = 1
my_ratings[793] = 5

Y, R = load_ratings_small()
Y = np.c_[my_ratings, Y]
R = np.c_[(my_ratings != 0).astype(int), R]
Ynorm, Ymean = normalizeRatings(Y, R)
num_movies, num_users = Y.shape

Gradient Descent using Tensorflow

In [7]:
tf.random.set_seed(1234)
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [8]:
iterations = 201
lambda_ = 1
for iter in range(iterations):
    with tf.GradientTape() as tape:
        cost_value = cofi_cost_func_vectorized(X, W, b, Ynorm, R, lambda_)
    grads = tape.gradient( cost_value, [X,W,b] )
    optimizer.apply_gradients( zip(grads, [X,W,b]) )
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 267453.3
Training loss at iteration 20: 16320.3
Training loss at iteration 40: 9551.2
Training loss at iteration 60: 7243.0
Training loss at iteration 80: 6215.2
Training loss at iteration 100: 5690.9
Training loss at iteration 120: 5412.0
Training loss at iteration 140: 5251.8
Training loss at iteration 160: 5147.2
Training loss at iteration 180: 5072.2
Training loss at iteration 200: 5015.0


In [9]:
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()
pm = p + Ymean
my_predictions = pm[:, 0]
ix = tf.argsort(my_predictions, direction='DESCENDING')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f}')




Original vs Predicted ratings:

Original 5.0, Predicted 4.79
Original 5.0, Predicted 4.71
Original 2.0, Predicted 2.57
Original 5.0, Predicted 4.86
Original 5.0, Predicted 4.77
Original 5.0, Predicted 4.59
Original 3.0, Predicted 3.38
Original 5.0, Predicted 4.51
Original 2.0, Predicted 2.35
Original 5.0, Predicted 4.66
Original 3.0, Predicted 3.33
Original 1.0, Predicted 1.30
Original 1.0, Predicted 1.19
